<a href="https://colab.research.google.com/github/miarpe/TFG/blob/main/TFG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TFG

##Web Scraping

###Dudas
Sólo los líderes europeos??
Recorrerlo y coger cada politico https://www.cidob.org/biografias_lideres_politicos/europa

In [2]:
!pip install requests



In [3]:
!pip install beautifulsoup4

In [4]:
import numpy as np
import pandas as pd
import requests
import bs4
import lxml.etree as xml

In [5]:
URL = "https://www.cidob.org/biografias_lideres_politicos/europa/moldova/maia_sandu"

requests.get(URL)

<Response [200]>

In [ ]:
web_page = bs4.BeautifulSoup(requests.get(URL, {}).text, "lxml")
web_page

###Final (Texto actualizado hasta diciembre 2020)??

In [7]:
index = web_page.find_all(name="div", attrs={"class": "ezxmltext-field"})

In [8]:
presentacion = index[0]
presentacion.text.replace("\n", "")

'En 2016 en Moldova, Maia Sandu, candidata centrista y proeuropea del Partido de Acción y Solidaridad (PAS), contendió sin éxito en una bronca elección presidencial con  Igor Dodon, líder del izquierdista y prorruso Partido de los Socialistas de la República de Moldova (PSRM). Tres años después, en junio de 2019, ambos, ella como cabeza de la tercera fuerza del Parlamento, la alianza ACUM formada por el PAS y la Plataforma Dignidad y Verdad, y él como presidente de la República y líder de facto del partido más votado en los comicios de febrero, suspendieron su mutua malquerencia en aras de un Gobierno de coalición mayoritario y con Sandu de primera ministra. Su nombramiento generó una enrevesada trifulca, pronto solventada, entre partidos e instituciones, viviendo el país durante unos días una auténtica crisis constitucional. Sin embargo, el inesperado entendimiento entre Sandu y Dodon, nunca convincente, no tardó en saltar en pedazos: al cabo de cinco meses, los socialistas declararon

In [9]:
biografia = index[1]
biografia.text.replace("\n", "")

'1. Una economista liberal decepcionada por la corrupción de la clase política moldava2. Candidata presidencial en 2016 enfrentada al socialista Dodon y primera ministra en 2019 en coalición con su partido3. Fallido experimento de gobierno revancha en las presidenciales de 20201. Una economista liberal decepcionada por la corrupción de la clase política moldava    Maia Sandu vivió el proceso histórico (1990-1992) de la soberanía y la independencia nacionales de su país a partir de la URSS siendo estudiante de Administración en la Academia de Estudios Económicos de Moldova (ASEM), centro universitario público de Chisinau. En 1994, con 22 años, entró a trabajar como consultora y funcionaria en el Ministerio de Economía, si bien continuó con su formación en las aulas de la Academia de Administración Pública (APA), que le otorgó un máster en Relaciones Internacionales en 1998. En la década siguiente, Sandu ejerció sucesivamente de economista en las oficinas del Banco Mundial en Moldova, di

In [10]:
import re

twitter = web_page.find(name="a", attrs={"href": re.compile("https://twitter.com/+")})
twitter["href"]


'https://twitter.com/sandumaiamd'

In [11]:
import re

facebook = web_page.find(name="a", attrs={"href": re.compile("https://www.facebook.com/+")})
facebook["href"]

'https://www.facebook.com/maia.sandu/'

In [12]:
import re

youtube = web_page.find(name="a", attrs={"href": re.compile("https://www.youtube.com/+")})
youtube["href"]

'https://www.youtube.com/results?search_query=maia+sandu'

In [13]:
import re

instagram = web_page.find(name="a", attrs={"href": re.compile("https://www.instagram.com/+")})
instagram["href"]

'https://www.instagram.com/maia.sandu/'

In [26]:
#filas = []    
#filas.append((presentacion.text.replace("\n", ""), biografia.text.replace("\n", ""), twitter["href"],facebook["href"],youtube["href"],instagram["href"]))

#columnas = []
#columnas.append(("index","presentacion", "biografia", "twitter","facebook","youtube","instagram"))

#df = pd.DataFrame(filas)

df = pd.DataFrame({'presentacion': [presentacion.text.replace("\n", "")], 'biografia': [biografia.text.replace("\n", "")], 'twitter': [twitter["href"]], 'facebook': [facebook["href"]], 'youtube': [youtube["href"]], 'instagram': [instagram["href"]] })  

df

,presentacion,biografia,twitter,facebook,youtube,instagram
0,"En 2016 en Moldova, Maia Sandu, candidata cent...",1. Una economista liberal decepcionada por la ...,https://twitter.com/sandumaiamd,https://www.facebook.com/maia.sandu/,https://www.youtube.com/results?search_query=m...,https://www.instagram.com/maia.sandu/


###Pasando a csv

In [28]:
df.to_csv('cidob.csv',index=False)

In [32]:
cidob = pd.read_csv('cidob.csv')
cidob

,presentacion,biografia,twitter,facebook,youtube,instagram
0,"En 2016 en Moldova, Maia Sandu, candidata cent...",1. Una economista liberal decepcionada por la ...,https://twitter.com/sandumaiamd,https://www.facebook.com/maia.sandu/,https://www.youtube.com/results?search_query=m...,https://www.instagram.com/maia.sandu/
